<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/langchain_MISTRAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U langchain-google-genai -q
!pip install colab-env --quiet

In [ ]:
!pip install huggingface_hub
!pip install transformers

In [ ]:
%pip install --upgrade --quiet  text-generation transformers google-search-results numexpr langchainhub sentencepiece jinja2

In [ ]:
!pip install langchain_community -q

In [ ]:
!pip install colab-env --quiet

In [8]:
from google.colab import drive

drive.mount('/content/gdrive',readonly=False,force_remount=True)

#!pip install diffusers safetensors  --quiet

import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")


from huggingface_hub import login

login(
  token=access_token_write,
  add_to_git_credential=True
)

Mounted at /content/gdrive
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
%pip install mistral_inference -q

In [41]:
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    #repo_id="mistralai/Mistral-7B-Instruct-v0.1",
    repo_id='mistralai/Mistral-7B-Instruct-v0.3',
    task="text-generation",
    huggingfacehub_api_token=access_token_write,
    model_kwargs={
        "max_new_tokens": 512,
        "top_k": 30,
        "temperature": 0.1,
        "repetition_penalty": 1.03,
    },
)

In [95]:
response = llm('The capital city of France')

In [96]:
print(response)

The capital city of France, Paris is one of the most popular tourist destinations in the world. It is known for its rich history, culture, art, and architecture. The city is home to some of the most famous landmarks in the world, including the Eiffel Tower, the Louvre Museum, and Notre-Dame Cathedral.

Paris is also known for its fashion industry, with many famous designers based in the city. The Champs-Élysées is one of the most famous shopping streets in the world, and the city is home to many luxury boutiques and designer stores.

In addition to its cultural and shopping attractions, Paris is also known for its food and wine. The city is home to many Michelin-starred restaurants, and it is famous for its café culture. Parisians are known for their love of coffee and pastries, and there are countless cafes and bakeries throughout the city where you can enjoy a croissant or a cup of coffee.

Paris is also known for its nightlife, with many bars, clubs, and live music venues throughout

In [ ]:
!pip install openai -q

In [ ]:
import colab_env
import openai
import os
openai.api_key = os.getenv("OPENAI_API_KEY")
%cd /content/
!wget "https://www.dropbox.com/s/f6bmb19xdg0xedm/paul_graham_essay.txt?dl=1" -O pg_essay.txt
#!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt' -O pg_essay.txt

In [ ]:
!pip install langchain -q
!pip install langchain-core -q
!pip install langchain-community -q
!pip install langchain-anthropic -q

In [14]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader('/content/pg_essay.txt')
data = loader.load()

In [15]:
from langchain_text_splitters import CharacterTextSplitter

splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = splitter.split_documents(data)

In [ ]:
!pip install tiktoken -q

In [ ]:
!pip install faiss-gpu -q
!pip install faiss-cpu -q

In [91]:
#from langchain_community.embeddings import CohereEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS


embeddings = OpenAIEmbeddings(model='text-embedding-3-small')
vector = FAISS.from_documents(documents, embeddings)

In [97]:
from langchain.chains import LLMChain, SimpleSequentialChain, RetrievalQA, ConversationalRetrievalChain
retriever = vector.as_retriever()

# create a chain to answer questions
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True)

In [93]:
#prompt="How AWS has evolved?"
prompt="What did the author do growing up?"
result = qa({"query": prompt})

In [94]:
print(result["result"])

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

What I Worked On

February 2021

Before college the two main things I worked on, outside of school, were writing and programming. I didn't write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.

The first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district's 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain's lair down there, with all these alien-looking machines — CPU, disk drives, printer, card reader — sitting up on a 

In [75]:
print()
#print('chain to answer questions')
print("-" * 80)
print()
result = qa({"query": prompt})
print(f'Query: {result["query"]}\n')
print(f'Result: {result["result"]}\n')
print(f'Context Documents: ')
for srcdoc in result["source_documents"]:
      print(f'{srcdoc}\n')
print("-" * 80)


--------------------------------------------------------------------------------

Query: What did the author do growing up?

Result: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

What I Worked On

February 2021

Before college the two main things I worked on, outside of school, were writing and programming. I didn't write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.

The first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district's 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a 